# 🧠 Machine Learning Models - Vanilla Price Prediction

Ce notebook implémente plusieurs modèles de prédiction:
1. **Baseline**: Moyenne mobile, ARIMA
2. **Machine Learning**: Random Forest, XGBoost
3. **Deep Learning**: Prophet (Facebook)
4. **Évaluation et comparaison**

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Time series
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

# Paths
DATA_PATH = Path('../data/processed')
MODEL_PATH = Path('../models')
OUTPUT_PATH = Path('../outputs/figures')

# Config
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("✅ Imports successful")

In [ ]:
# Charger les données
df = pd.read_csv(DATA_PATH / 'vanilla_prices_clean.csv', parse_dates=['date'])
df = df.set_index('date')

print(f"📊 Dataset: {len(df)} observations")
print(f"📅 Période: {df.index.min().date()} → {df.index.max().date()}")
df.head()

## 1. Préparation des données

In [ ]:
# Features et target
target = 'price_usd_kg'

features = [
    'year', 'month', 'quarter',
    'harvest_season', 'cyclone_season',
    'price_lag1', 'price_lag3', 'price_lag6', 'price_lag12',
    'price_ma3', 'price_ma6', 'price_ma12',
    'price_pct_change', 'price_volatility'
]

X = df[features]
y = df[target]

print(f"Features: {len(features)}")
print(f"Observations: {len(X)}")

In [ ]:
# Split temporel (80% train, 20% test)
# IMPORTANT: Pour les séries temporelles, on ne fait PAS de shuffle!

train_size = int(len(df) * 0.8)
train_idx = df.index[:train_size]
test_idx = df.index[train_size:]

X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

print(f"📊 Train: {len(X_train)} observations ({df.index[0].date()} → {df.index[train_size-1].date()})")
print(f"📊 Test: {len(X_test)} observations ({df.index[train_size].date()} → {df.index[-1].date()})")

In [ ]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled")

## 2. Fonctions d'évaluation

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """Calcule et affiche les métriques d'évaluation"""
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n📊 {model_name} - Résultats:")
    print(f"   RMSE: ${rmse:.2f}")
    print(f"   MAE:  ${mae:.2f}")
    print(f"   MAPE: {mape:.2f}%")
    print(f"   R²:   {r2:.4f}")
    
    return {'model': model_name, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

def plot_predictions(y_true, y_pred, dates, model_name):
    """Visualise les prédictions vs réalité"""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Time series plot
    ax1 = axes[0]
    ax1.plot(dates, y_true, 'b-', label='Réel', linewidth=2)
    ax1.plot(dates, y_pred, 'r--', label='Prédit', linewidth=2)
    ax1.fill_between(dates, y_true, y_pred, alpha=0.3, color='gray')
    ax1.set_title(f'{model_name} - Prédictions vs Réalité', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Prix (USD/kg)')
    ax1.legend()
    
    # Scatter plot
    ax2 = axes[1]
    ax2.scatter(y_true, y_pred, alpha=0.6, edgecolors='black')
    ax2.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', linewidth=2)
    ax2.set_title(f'{model_name} - Scatter Plot', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Prix Réel (USD/kg)')
    ax2.set_ylabel('Prix Prédit (USD/kg)')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_PATH / f'{model_name.lower().replace(" ", "_")}_predictions.png', dpi=150)
    plt.show()

# Stocker les résultats
results = []

## 3. Modèle Baseline - Moyenne Mobile

In [ ]:
# Baseline: prédire avec la moyenne mobile des 3 derniers mois
y_pred_baseline = X_test['price_ma3'].values

result = evaluate_model(y_test.values, y_pred_baseline, 'Baseline (MA3)')
results.append(result)

plot_predictions(y_test.values, y_pred_baseline, test_idx, 'Baseline MA3')

## 4. SARIMA (Seasonal ARIMA)

In [ ]:
# Auto-ARIMA pour trouver les meilleurs paramètres
print("🔍 Recherche des paramètres optimaux SARIMA...")

auto_arima = pm.auto_arima(
    y_train,
    seasonal=True,
    m=12,  # Saisonnalité mensuelle
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    max_p=3, max_q=3,
    max_P=2, max_Q=2,
    trace=True
)

print(f"\n✅ Meilleur modèle: {auto_arima.summary()}")

In [ ]:
# Prédictions SARIMA
y_pred_sarima = auto_arima.predict(n_periods=len(y_test))

result = evaluate_model(y_test.values, y_pred_sarima, 'SARIMA')
results.append(result)

plot_predictions(y_test.values, y_pred_sarima, test_idx, 'SARIMA')

## 5. Random Forest

In [ ]:
# Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

result = evaluate_model(y_test.values, y_pred_rf, 'Random Forest')
results.append(result)

plot_predictions(y_test.values, y_pred_rf, test_idx, 'Random Forest')

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('Importance')
plt.title('Random Forest - Importance des Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'rf_feature_importance.png', dpi=150)
plt.show()

## 6. XGBoost

In [ ]:
# XGBoost Regressor
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=0
)

xgb_model.fit(
    X_train_scaled, y_train,
    eval_set=[(X_test_scaled, y_test)],
    verbose=False
)

y_pred_xgb = xgb_model.predict(X_test_scaled)

result = evaluate_model(y_test.values, y_pred_xgb, 'XGBoost')
results.append(result)

plot_predictions(y_test.values, y_pred_xgb, test_idx, 'XGBoost')

## 7. Prophet (Facebook)

In [ ]:
# Prophet nécessite un format spécifique
try:
    from prophet import Prophet
    
    # Préparer les données pour Prophet
    df_prophet_train = pd.DataFrame({
        'ds': train_idx,
        'y': y_train.values
    })
    
    df_prophet_test = pd.DataFrame({
        'ds': test_idx
    })
    
    # Créer et entraîner le modèle
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05
    )
    prophet_model.fit(df_prophet_train)
    
    # Prédictions
    forecast = prophet_model.predict(df_prophet_test)
    y_pred_prophet = forecast['yhat'].values
    
    result = evaluate_model(y_test.values, y_pred_prophet, 'Prophet')
    results.append(result)
    
    plot_predictions(y_test.values, y_pred_prophet, test_idx, 'Prophet')
    
except ImportError:
    print("⚠️ Prophet non installé. Exécuter: pip install prophet")
    print("   Skipping Prophet model...")

## 8. Comparaison des modèles

In [ ]:
# Tableau comparatif
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('RMSE')

print("\n" + "="*60)
print("📊 COMPARAISON DES MODÈLES")
print("="*60)
print(results_df.to_string(index=False))
print("\n🏆 Meilleur modèle:", results_df.iloc[0]['model'])

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['RMSE', 'MAE', 'MAPE', 'R2']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for i, (metric, ax) in enumerate(zip(metrics, axes.flat)):
    values = results_df[metric].values
    models = results_df['model'].values
    
    bars = ax.barh(models, values, color=colors[i], edgecolor='black')
    ax.set_xlabel(metric)
    ax.set_title(f'Comparaison - {metric}', fontweight='bold')
    
    # Annoter les valeurs
    for bar, val in zip(bars, values):
        if metric == 'MAPE':
            ax.text(val + 0.5, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center')
        elif metric == 'R2':
            ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center')
        else:
            ax.text(val + 1, bar.get_y() + bar.get_height()/2, f'${val:.1f}', va='center')

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Prédictions futures

In [ ]:
# Utiliser le meilleur modèle pour prédire 12 mois dans le futur
best_model_name = results_df.iloc[0]['model']
print(f"\n🔮 Prédictions avec {best_model_name} pour les 12 prochains mois:")

# Générer les prédictions avec SARIMA (plus adapté pour forecast futur)
future_predictions = auto_arima.predict(n_periods=12)
future_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=12, freq='MS')

future_df = pd.DataFrame({
    'Date': future_dates,
    'Prix Prédit (USD/kg)': future_predictions
})

print(future_df.to_string(index=False))

In [ ]:
# Visualisation des prédictions futures
plt.figure(figsize=(14, 6))

# Données historiques
plt.plot(df.index, df['price_usd_kg'], 'b-', label='Historique', linewidth=2)

# Prédictions futures
plt.plot(future_dates, future_predictions, 'r--', label='Prédictions 2025', linewidth=2, marker='o')

# Zone de confiance (approximative)
std = df['price_usd_kg'].std() * 0.3
plt.fill_between(future_dates, 
                 future_predictions - std, 
                 future_predictions + std, 
                 alpha=0.3, color='red', label='Intervalle de confiance')

plt.axvline(x=df.index[-1], color='gray', linestyle='--', alpha=0.5)
plt.title('Prédiction du Prix de la Vanille - 12 Prochains Mois', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Prix (USD/kg)')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'future_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Sauvegarde des modèles

In [ ]:
import joblib

# Sauvegarder les modèles
joblib.dump(rf_model, MODEL_PATH / 'random_forest_model.joblib')
joblib.dump(xgb_model, MODEL_PATH / 'xgboost_model.joblib')
joblib.dump(scaler, MODEL_PATH / 'scaler.joblib')
joblib.dump(auto_arima, MODEL_PATH / 'sarima_model.joblib')

# Sauvegarder les résultats
results_df.to_csv(MODEL_PATH / 'model_results.csv', index=False)

print("✅ Modèles sauvegardés dans models/")
print("   - random_forest_model.joblib")
print("   - xgboost_model.joblib")
print("   - sarima_model.joblib")
print("   - scaler.joblib")
print("   - model_results.csv")

## 📋 Résumé

### Modèles testés:
1. Baseline (Moyenne Mobile 3 mois)
2. SARIMA (Auto-tuned)
3. Random Forest
4. XGBoost
5. Prophet (si installé)

### Prochaines améliorations possibles:
- Hyperparameter tuning avec GridSearchCV
- Ensemble methods (stacking)
- Ajouter features externes (taux de change, météo)
- LSTM pour deep learning

### Utilisation:
```python
import joblib
model = joblib.load('models/xgboost_model.joblib')
scaler = joblib.load('models/scaler.joblib')
prediction = model.predict(scaler.transform(new_data))
```